# Precompiling Numba modules

One issue with Numba is that it can be hard to install.  With `conda` it's incredibly easy, but not everybody uses `conda` and trying to explain to users/collaborators why they're doing everything wrong is hard.  

Tools like SWIG can compile C/C++ (or other) code at install time and make it available as a Python module if there's some serious numerical heavy-lifting required.  

But if you have ever tried to use SWIG together with NumPy... suffice to say it's a less than ideal arrangement (please don't hurt me, @dabeaz).

## Ahead-of-Time compilation

While Numba's main use is in JIT compiling, they do provide tools for doing AOT compilation.  This pre-compiled module does not rely on Numba, only on NumPy.  (If you are working with collaborators who don't have NumPy installed, I can't help you).

We need to import `numpy`, of course, and also `numba.pycc.CC`

In [9]:
import numpy
from math import sqrt
from numba.pycc import CC

Name the module `ppe` (I am not creative)

In [10]:
cc = CC('ppe')
cc.verbose = True

In [11]:
@cc.export('pressure_poisson', 
           'f8[:,:](f8[:,:], f8[:,:], f8)')
def pressure_poisson(p, b, l2_target):
    I, J = b.shape

    iter_diff = l2_target + 1

    n = 0
    while iter_diff > l2_target and n <= 500:
        pn = p.copy()
        for i in range(1, I - 1):
            for j in range(1, J - 1):
                p[i, j] = (.25 * (pn[i, j + 1] +
                                  pn[i, j - 1] +
                                  pn[i + 1, j] +
                                  pn[i - 1, j]) -
                                  b[i, j])

        for i in range(I):
            p[i, 0] = p[i, 1]
            p[i, -1] = 0

        for j in range(J):
            p[0, j] = p[1, j]
            p[-1, j] = p[-2, j]

        if n % 10 == 0:
            iter_diff = sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))

        n += 1

    return p


In [12]:
@cc.export('test', '()')
def test():
    print("I'm inside a .so file, send help!")

**Note:** Each function in the module can be compiled with one type signature only.  You can specify multiple types, each with its own function name, e.g.

```python
@cc.export('pressure_poisson_single', 
           'f4[:,:](f4[:,:], f4[:,:], f4)')
@cc.export('pressure_poisson_double', 
           'f8[:,:](f8[:,:], f8[:,:], f8)')
@cc.export('pressure_poisson_quad', 
           'f16[:,:](f16[:,:], f16[:,:], f16)')
def pressure_poisson(p, b, l2_target=1e-4):
```

In [13]:
cc.compile()

generating LLVM code for 'ppe' into /tmp/pycc-build-ppe-df6bt4cz/ppe.cpython-36m-x86_64-linux-gnu.o
C compiler: gcc -pthread -B /home/martinci/.local/share/anaconda3/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC

creating /tmp/pycc-build-ppe-df6bt4cz/home
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.local
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.local/share
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.local/share/anaconda3
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.local/share/anaconda3/lib
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.local/share/anaconda3/lib/python3.6
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.local/share/anaconda3/lib/python3.6/site-packages
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.local/share/anaconda3/lib/python3.6/site-packages/numba
creating /tmp/pycc-build-ppe-df6bt4cz/home/martinci/.loc

In [14]:
%ls

01.When.where.to.use.Numba.ipynb
02.Intro.to.jit.ipynb
03.How.Numba.Works.ipynb
04.Direct.Summation.ipynb
05.0.A.Breakneck.Introduction.to.CFD.ipynb
05.1.Cavity_Flow.ipynb
05.2.optional.Numba.v.Cython.v.Fortran.ipynb
06.1.optional.Compile.Module.ipynb
06.2.optional.Test.Compiled.Module.ipynb
07.Make.your.own.ufuncs.ipynb
08.Make.generalized.ufuncs.ipynb
09.Tips.and.FAQ.ipynb
10.optional.Numba.and.ipyparallel.ipynb
11.optional.jit.parallel.ipynb
bad_script.py
exercises/
fail.html
figures/
IC.pickle
loop.lifting.ipynb
nopython_failure.py
numpy_ans.pickle
ppe.cpython-36m-x86_64-linux-gnu.so*
snippets/
timings.txt


In [15]:
from ppe import test

In [16]:
test()

I'm inside a .so file. Send help!
